In [1]:
from __future__ import division

import pandas as pd
import numpy as np
import math
from datetime import datetime,timedelta

%matplotlib inline

In [2]:
training_path = 'dataSets/training/'
test_path1 = 'dataSets/testing_phase1/'

In [3]:
training_traj = pd.read_csv(training_path+'trajectories(table 5)_training.csv')

In [4]:
test_traj1 = pd.read_csv(test_path1+'trajectories(table 5)_test1.csv')

In [5]:
def time_to_window(x):
    dt = datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
    dtmin = int(dt.minute / 20) * 20
    dtwindow = datetime(dt.year, dt.month, dt.day, dt.hour, dtmin, 0)
    return dtwindow

In [21]:
def format_window(wd):
    return '[{},{})'.format(str(wd), str(wd+timedelta(minutes=20)))

In [22]:
#training_traj['starting_datetime'] = pd.to_datetime(training_path['starting_time'])
training_traj['window'] = training_traj['starting_time'].apply(time_to_window)

training_traj_group_mean = \
training_traj.groupby(['intersection_id','tollgate_id','window']).mean()['travel_time']

training_traj_group_count = \
training_traj.groupby(['intersection_id','tollgate_id','window']).count()['travel_time']

In [23]:
ss = pd.read_csv('submission/submission_sample_travelTime.csv')

In [24]:
print(ss['time_window'][0])
print(format_window(training_traj['window'][0]))

[2016-10-18 08:00:00,2016-10-18 08:20:00)
[2016-07-19 00:00:00,2016-07-19 00:20:00)


In [25]:
from datetime import date, time

In [48]:
date_list = []

d1 = date(2016, 7, 19)  # start date
d2 = date(2016, 10, 17)  # end date

delta = d2 - d1         # timedelta

for i in range(delta.days + 1):
    date_list.append(d1 + timedelta(days=i))

give_time_list1 = [time(6,0), time(6,20), time(6,40), 
                   time(7,0), time(7,20), time(7,40)]
pred_time_list1 = [time(8,0), time(8,20), time(8,40), 
                   time(9,0), time(9,20), time(9,40)]

give_time_list2 = [time(15,0), time(15,20), time(15,40), 
                   time(16,0), time(16,20), time(16,40)]
pred_time_list2 = [time(17,0), time(17,20), time(17,40), 
                   time(18,0), time(18,20), time(18,40)]


In [83]:
date_list2 = []

d1 = date(2016, 10, 11)  # start date
d2 = date(2016, 10, 17)  # end date

delta = d2 - d1         # timedelta

for i in range(delta.days + 1):
    date_list2.append(d1 + timedelta(days=i))


In [27]:
# A2m.index[300].weekday()

In [29]:
def interpolate_missing_window(x):
    ss = [(i-datetime(1970,1,1)).total_seconds() for i in x.index]
    tt = x.values
    ssall = np.linspace(ss[0], ss[-1], (ss[-1]-ss[0])/1200+1)
    print(len(ss), len(tt), len(ssall))
    ttall = np.interp(ssall, ss, tt)
    ssall = [int(i) for i in ssall]
    ssall = [datetime(1970,1,1)+timedelta(seconds=i) for i in ssall]
    return pd.Series(index=ssall, data=ttall)

In [28]:
A2m = training_traj_group_mean.loc['A'].loc[2]


In [30]:
A2m = interpolate_missing_window(A2m)


(5965, 5965, 6551)


In [90]:
from scipy.stats import gmean

In [105]:
morning_true = np.array([])
morning_pred = np.array([])
for ii, ti in [('A',2),('A',3),('B',1),('B',3),('C',1),('C',3)]:
    mtmp = training_traj_group_mean.loc[ii].loc[ti]
    mtmp = interpolate_missing_window(mtmp)
    test6 = []
    for i in date_list:
        tmp = []
        for j in pred_time_list2:
            tmp.append(mtmp[datetime.combine(i, j)])
        test6.append(tmp)
    result6 = test6
    morning_true = np.append(morning_true, result6)
    
    test6 = []
    for i in date_list:
        tmp = []
        for j in give_time_list2:
            tmp.append(mtmp[datetime.combine(i, j)])
        test6.append(tmp)
    result6 = np.repeat([np.mean(i) for i in test6], 6)
    morning_pred = np.append(morning_pred, result6) 
    

(5965, 5965, 6551)
(5304, 5304, 6548)
(3206, 3206, 6550)
(4803, 4803, 6551)
(3290, 3290, 6547)
(2576, 2576, 6547)


In [106]:
pred_loss(morning_pred, morning_true)

0.21667818913872719

In [31]:
morning_train66 = []
for i in date_list:
    tmp = []
    for j in give_time_list1:
        tmp.append(A2m[datetime.combine(i, j)])
    for j in pred_time_list1:
        tmp.append(A2m[datetime.combine(i, j)]) 
    morning_train66.append(tmp)
    

In [49]:
night_train66 = []
for i in date_list:
    tmp = []
    for j in give_time_list2:
        tmp.append(A2m[datetime.combine(i, j)])
    for j in pred_time_list2:
        tmp.append(A2m[datetime.combine(i, j)]) 
    night_train66.append(tmp)
    

In [33]:
def pred_loss(x_pred, x_true):
    x_pred = np.array(x_pred)
    x_true = np.array(x_true)
    return (abs(x_pred-x_true)/x_true).mean()


In [34]:
def naive_pred_loss(x):
    first6 = x[:6]
    pred = [np.mean(first6)]*6
    return pred_loss(pred, x[6:])

In [35]:
naive_loss_list = [naive_pred_loss(i) for i in morning_train66]


In [50]:
naive_loss_list = [naive_pred_loss(i) for i in night_train66]

In [52]:
np.mean(naive_loss_list)


0.22280558325098837

# Make first submission 

Most naive approach, use average of the former 2 hours for prediction.

In [37]:
#training_traj['starting_datetime'] = pd.to_datetime(training_path['starting_time'])
training_traj['window'] = training_traj['starting_time'].apply(time_to_window)

training_traj_group_mean = \
training_traj.groupby(['intersection_id','tollgate_id','window']).mean()['travel_time']

training_traj_group_count = \
training_traj.groupby(['intersection_id','tollgate_id','window']).count()['travel_time']

In [38]:
test_traj1['window'] = test_traj1['starting_time'].apply(time_to_window)
test_traj1_group_mean = \
test_traj1.groupby(['intersection_id','tollgate_id','window']).mean()['travel_time']



In [39]:
date_list = []

d1 = date(2016, 10, 18)  # start date
d2 = date(2016, 10, 24)  # end date

delta = d2 - d1         # timedelta

for i in range(delta.days + 1):
    date_list.append(d1 + timedelta(days=i))

give_time_list1 = [time(6,0), time(6,20), time(6,40), 
                   time(7,0), time(7,20), time(7,40)]
# pred_time_list1 = [time(8,0), time(8,20), time(8,40), 
#                    time(9,0), time(9,20), time(9,40)]
give_time_list2 = [time(15,0), time(15,20), time(15,40), 
                   time(16,0), time(16,20), time(16,40)]

In [40]:
A2mt = test_traj1_group_mean.loc['A'].loc[2]
A2mt = interpolate_missing_window(A2mt)

(83, 83, 465)


In [41]:
morning_test6 = []
for i in date_list:
    tmp = []
    for j in give_time_list1:
        tmp.append(A2mt[datetime.combine(i, j)])
    morning_test6.append(tmp)

In [42]:
tmp

[41.18833333333334,
 50.115714285714283,
 37.168000000000006,
 75.683750000000003,
 54.032857142857146,
 78.277999999999992]

In [43]:
morning = np.array([])
for ii, ti in [('A',2),('A',3),('B',1),('B',3),('C',1),('C',3)]:
    mtmp = test_traj1_group_mean.loc[ii].loc[ti]
    mtmp = interpolate_missing_window(mtmp)
    test6 = []
    for i in date_list:
        tmp = []
        for j in give_time_list1:
            tmp.append(mtmp[datetime.combine(i, j)])
        test6.append(tmp)
    result6 = np.repeat([np.mean(i) for i in test6], 6)
    morning = np.append(morning, result6)
    

(83, 83, 465)
(84, 84, 465)
(75, 75, 465)
(77, 77, 465)
(69, 69, 465)
(60, 60, 465)


In [44]:
night = np.array([])
for ii, ti in [('A',2),('A',3),('B',1),('B',3),('C',1),('C',3)]:
    mtmp = test_traj1_group_mean.loc[ii].loc[ti]
    mtmp = interpolate_missing_window(mtmp)
    test6 = []
    for i in date_list:
        tmp = []
        for j in give_time_list2:
            tmp.append(mtmp[datetime.combine(i, j)])
        test6.append(tmp)
    result6 = np.repeat([np.mean(i) for i in test6], 6)
    night = np.append(night, result6)

(83, 83, 465)
(84, 84, 465)
(75, 75, 465)
(77, 77, 465)
(69, 69, 465)
(60, 60, 465)


In [45]:
ss['avg_travel_time'] = np.append(morning, night)

In [47]:
ss.head(20)

,intersection_id,tollgate_id,time_window,avg_travel_time
0,A,2,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",54.195926
1,A,2,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",54.195926
2,A,2,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",54.195926
3,A,2,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",54.195926
4,A,2,"[2016-10-18 09:20:00,2016-10-18 09:40:00)",54.195926
5,A,2,"[2016-10-18 09:40:00,2016-10-18 10:00:00)",54.195926
6,A,2,"[2016-10-19 08:00:00,2016-10-19 08:20:00)",60.403236
7,A,2,"[2016-10-19 08:20:00,2016-10-19 08:40:00)",60.403236
8,A,2,"[2016-10-19 08:40:00,2016-10-19 09:00:00)",60.403236
9,A,2,"[2016-10-19 09:00:00,2016-10-19 09:20:00)",60.403236


In [110]:
ss.to_csv('submission/submission_0331.csv', index=False)

# Following is the script given

In [18]:
# -*- coding: utf-8 -*-
#!/usr/bin/env python

"""
Objective:
Calculate the average travel time for each 20-minute time window.

"""

# import necessary modules
import math
from datetime import datetime,timedelta

file_suffix = '.csv'
path = 'dataSets/training/'  # set the data directory

def avgTravelTime(in_file):

    out_suffix = '_20min_avg_travel_time'
    in_file_name = in_file + file_suffix
    out_file_name = in_file.split('_')[1] + out_suffix + file_suffix

    # Step 1: Load trajectories
    fr = open(path + in_file_name, 'r')
    fr.readline()  # skip the header
    traj_data = fr.readlines()
    fr.close()
    print(traj_data[0])

    # Step 2: Create a dictionary to store travel time for each route per time window
    travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
    for i in range(len(traj_data)):
        each_traj = traj_data[i].replace('"', '').split(',')
        intersection_id = each_traj[0]
        tollgate_id = each_traj[1]

        route_id = intersection_id + '-' + tollgate_id
        if route_id not in travel_times.keys():
            travel_times[route_id] = {}

        trace_start_time = each_traj[3]
        trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
        time_window_minute = int(trace_start_time.minute / 20) * 20
        start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
        tt = float(each_traj[-1]) # travel time

        if start_time_window not in travel_times[route_id].keys():
            travel_times[route_id][start_time_window] = [tt]
        else:
            travel_times[route_id][start_time_window].append(tt)

    # Step 3: Calculate average travel time for each route per time window
    fw = open(out_file_name, 'w')
    fw.writelines(','.join(['"intersection_id"', '"tollgate_id"', '"time_window"', '"avg_travel_time"']) + '\n')
    for route in travel_times.keys():
        route_time_windows = list(travel_times[route].keys())
        route_time_windows.sort()
        for time_window_start in route_time_windows:
            time_window_end = time_window_start + timedelta(minutes=20)
            tt_set = travel_times[route][time_window_start]
            avg_tt = round(sum(tt_set) / float(len(tt_set)), 2)
            out_line = ','.join(['"' + route.split('-')[0] + '"', '"' + route.split('-')[1] + '"',
                                 '"[' + str(time_window_start) + ',' + str(time_window_end) + ')"',
                                 '"' + str(avg_tt) + '"']) + '\n'
            fw.writelines(out_line)
    fw.close()

def main():

    in_file = 'trajectories(table 5)_training'
    avgTravelTime(in_file)

if __name__ == '__main__':
    main()





"B","3","1065642","2016-07-19 00:14:24","105#2016-07-19 00:14:24#9.56;100#2016-07-19 00:14:34#6.75;111#2016-07-19 00:14:41#13.00;103#2016-07-19 00:14:54#7.47;122#2016-07-19 00:15:02#32.85","70.85"

